In [1]:
import os
import sys
sys.path.append("/mofem_install/spack/opt/spack/linux-ubuntu20.04-x86_64/gcc-9.4.0/tfel-4.0.0-jjcwdu6cbil5dzqzjhjekn3jdzo3e6gc/lib/python3.11/site-packages")
import numpy as np
import mtest

m = mtest.MTest()
# mtest.setVerboseMode(mtest.VerboseLevel.VERBOSE_QUIET)
m.setMaximumNumberOfSubSteps(20)
m.setModellingHypothesis("Tridimensional")

model = "DruckerPragerSimple"


# Set  material model implementation and path
lib_path = "/mofem_install/jupyter/thomas/mfront_interface/src/libBehaviour.so"


b = mtest.Behaviour('generic', lib_path, model,'Tridimensional')
m.setBehaviour("generic", lib_path, model)
# in mfront gallery:
# tan(beta) * p + q - R_DP_0 = 0
#R_DP_0 is named d in gallery example

# in my own implementation
# - M_JP * p' + J - M_JP * c' / tan(phi) = 0
# J = q / sqrt(3)
# q = sqrt(3J2)

E = 150
nu = 0.3
# phi is the internal friction angle
phi = np.radians(35)
c = 30

# fitting at triaxial compression: lode angle = -30
# M_JP = 2 * np.sqrt(3) * np.sin(phi) / (3 - np.sin(phi))
# d = M_JP * c / np.tan(phi)

# Loading programme
tMax = 1.0  # s , total time
nTime = 200
ltime = np.linspace(0.0, tMax, nTime)
# Environment parameters
m.setExternalStateVariable("Temperature", 293.15)
# Material parameters
m.setMaterialProperty("YoungModulus", E)
m.setMaterialProperty("PoissonRatio", nu)
m.setMaterialProperty("phi", phi)
m.setMaterialProperty("c", c)

controls = ["stress", "strain"]
control = controls[0]
if control == "stress":
    p_con_0 =  0 # confining pressure
    p_con_1 =  00 # confining pressure
    p_axi_0 = 0
    p_axi_1 = 45
    m.setImposedStress("SXX", {0: p_con_0, 0.02: p_con_0, 1.0: p_con_1})
    m.setImposedStress("SYY", {0: p_con_0, 0.02: p_con_0, 1.0: p_con_1})
    m.setImposedStress("SZZ", {0: p_axi_0, 0.02: p_axi_0, 1.0: p_axi_1})
if control == "strain":
    # Set imposed strains for axial and confining directions
    m.setImposedStrain("EXX", {0: 0, 1.0: 0.00001})
    m.setImposedStrain("EYY", {0: 0, 1.0: 0.00001})
    m.setImposedStrain("EZZ", {0: 0, 1.0: 2.7})


s = mtest.MTestCurrentState()
wk = mtest.MTestWorkSpace()
m.completeInitialisation()
m.initializeCurrentState(s)
m.initializeWorkSpace(wk)


# run sim
for i in range(nTime - 1):
    # print(f"===========Loop {i}===========")
    m.execute(s, wk, ltime[i], ltime[i + 1])


** no prediction
resolution from 0 to 0.00502513
iteration 1 : 0 0 (0 0 0 0 0 0)
iteration 2 : 0 0 (0 0 0 0 0 0)
convergence, after 2 iterations, order undefined

resolution from 0.00502513 to 0.0100503
iteration 1 : 0 0 (0 0 0 0 0 0)
iteration 2 : 0 0 (0 0 0 0 0 0)
convergence, after 2 iterations, order undefined

resolution from 0.0100503 to 0.0150754
iteration 1 : 0 0 (0 0 0 0 0 0)
iteration 2 : 0 0 (0 0 0 0 0 0)
convergence, after 2 iterations, order undefined

resolution from 0.0150754 to 0.0201005
iteration 1 : 3.07661e-05 0.00461491 (-9.22982e-06 -9.22982e-06 3.07661e-05 0 0 0)
iteration 2 : 4.92417e-18 1.00961e-15 (-9.22982e-06 -9.22982e-06 3.07661e-05 0 0 0)
convergence, after 2 iterations, order undefined

resolution from 0.0201005 to 0.0251256
iteration 1 : 0.0015383 0.230746 (-0.000470721 -0.000470721 0.00156907 0 0 0)
iteration 2 : 9.13578e-15 1.80336e-12 (-0.000470721 -0.000470721 0.00156907 0 0 0)
convergence, after 2 iterations, order undefined

resolution from 0.025125

RuntimeError: GenericSolver::execute: maximum number of sub stepping reached

In [ ]:
import gmsh
from pydantic import BaseModel
import os
import sys
sys.path.append('/mofem_install/jupyter/thomas/mfront_example_test/src')

import mesh_create as mshcrte
import custom_models as cm
import utils as ut
import plotting

os.chdir("/mofem_install/jupyter/thomas/mfront_example_test")
class Box(BaseModel):
    x: float = 0
    y: float = 0
    z: float = 0
    dx: float = 80
    dy: float = 80
    dz: float = 29.5

    def create(self):
        
        gmsh.initialize()
        gmsh.option.setNumber("General.Verbosity", 99)
        
        pt1 = gmsh.model.occ.addPoint(self.x, self.y, self.z)
        pt2 = gmsh.model.occ.addPoint(self.x + self.dx , self.y, self.z)
        pt3 = gmsh.model.occ.addPoint(self.x + self.dx , self.y + self.dy, self.z)
        cpt1 = gmsh.model.occ.addPoint(41,40,0)
        cpt2 = gmsh.model.occ.addPoint(40,39,0)
        cpt3 = gmsh.model.occ.addPoint(39,40,0)
        pt4 = gmsh.model.occ.addPoint(self.x, self.y + self.dy, self.z)

        l1 = gmsh.model.occ.addLine(pt1, pt2)
        l2 = gmsh.model.occ.addLine(pt2, pt3)
        l3 = gmsh.model.occ.addLine(pt3, cpt3)
        cA1 = gmsh.model.occ.addCircleArc(cpt1,cpt2,cpt3)
        l4 = gmsh.model.occ.addLine(cpt1, pt4)
        l5 = gmsh.model.occ.addLine(pt4, pt1)
        cl1 = gmsh.model.occ.addCurveLoop([l1,l2,l3, cA1, l4, l5])
        s1 = gmsh.model.occ.addPlaneSurface([cl1])
        # total_depth = 20
        volume = gmsh.model.occ.extrude([(2, s1)], 0, 0, 29.5, [2], recombine=True)
        print(volume)
        for i in volume:
            if i[0] == 3:
                dimTag = i[1]
                break
        test = mshcrte.get_surface_extremes(dimTag)
        print(test)
        volume2 = gmsh.model.occ.extrude([(2, test.max_z_surfaces[0])], 0, 0, 14.1, [2], recombine=True)
        for i in volume:
            if i[0] == 3:
                dimTag2 = i[1]
                break
            
        
        
        # pile_manager = cm.PileManager(x=0, y=0, z=29.5+14.1, dx=0, dy=0, dz=-20.5, R=1, r=0.975,
        #                         preferred_model= cm.PropertyTypeEnum.elastic,
        #                         props = {cm.PropertyTypeEnum.elastic: cm.LinearElasticProperties(youngs_modulus=200000*(10**6), poisson_ratio=0.3)},
        #                         )
        
        # outer_cylinder_tag, inner_cylinder_tag = pile_manager.addPile()
        
        # cut_soil_tags, _ = gmsh.model.occ.cut(
        #     [[3, tag] for tag in [dimTag, dimTag2]],
        #     [[3, outer_cylinder_tag]],
        #     -1,
        #     removeObject=True,
        #     removeTool=False,
        # )
        # cut_soil_boxes = [tag for _, tag in cut_soil_tags]

        # symmetry_cutter_tag = gmsh.model.occ.addBox(-200, 0, -200, 400, 400, 400)

        # # Cut the outer cylinder with the inner cylinder to form the pile
        # cut_pile_tags, _ = gmsh.model.occ.cut(
        #     [[3, outer_cylinder_tag]],
        #     [[3, inner_cylinder_tag]],
        #     -1,
        #     removeObject=True,
        #     removeTool=True,
        # )
        # pile_vols = [tag for dim, tag in cut_pile_tags]
        
        
        gmsh.model.occ.synchronize()
        for curve in [l1,l2,l3,l4]:
            gmsh.model.mesh.setTransfiniteCurve(curve,10)
            
        for surf in gmsh.model.getEntities(2):
            gmsh.model.mesh.setTransfiniteSurface(surf[1])
        
        for vol in gmsh.model.getEntities(3):
            gmsh.model.mesh.setTransfiniteVolume(vol[1])
        
        gmsh.option.setNumber('Mesh.RecombineAll', 1)
        gmsh.option.setNumber('Mesh.Recombine3DAll', 1)
        gmsh.option.setNumber('Mesh.RecombinationAlgorithm', 2)
        # gmsh.option.setNumber('Mesh.Recombine3DLevel', 0)
        gmsh.option.setNumber('Mesh.SaveAll', 1)
        # gmsh.option.setNumber('Mesh.SaveGroupsOfNodes', 1)
        # gmsh.option.setNumber('Mesh.SecondOrderIncomplete', 1)
        # gmsh.option.setNumber("Mesh.MeshSizeMax", 10)
        # gmsh.option.setNumber("Mesh.MeshSizeMax", 0.5)


        gmsh.model.mesh.generate(3)
        gmsh.model.mesh.recombine()
        gmsh.write("testing.med")
        gmsh.finalize()
    
    
Box().create()